In [1]:
!pip install -q transformers sentence-transformers gradio datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00


In [16]:
from google.colab import files

#enterprise_data.db
import pandas as pd
import sqlite3


conn = sqlite3.connect("enterprise_data.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)



Tables in database: [('branch_revenue',)]


In [17]:
df_new = pd.DataFrame([
    {'branch_name': 'Miami Branch', 'revenue': 72000.00, 'quarter': 'Q2', 'year':2025},
    {'branch_name': 'Dallas Branch', 'revenue': 88000.00, 'quarter': 'Q2', 'year':2025},
    {'branch_name': 'Phoenix Branch', 'revenue': 188000.00, 'quarter': 'Q3', 'year':2025},
    {'branch_name': 'Austin Branch', 'revenue': 128000.00, 'quarter': 'Q3', 'year':2025},
    {'branch_name': 'Denver Branch', 'revenue': 126000.00, 'quarter': 'Q3', 'year':2025}
])

df_new.to_sql('branch_revenue', conn, if_exists='append', index=False)


5

In [18]:

from sentence_transformers import SentenceTransformer, util

# Connect to uploaded SQLite file
conn = sqlite3.connect("enterprise_data.db")  # Replace with your file name
cursor = conn.cursor()

# Load table data
df = pd.read_sql_query("SELECT * FROM branch_revenue", conn)
print(df)
conn.close()

# Embed rows
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus = df.astype(str).agg(" ".join, axis=1).tolist()
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)


      branch_name quarter  year    revenue
0         Phoenix      Q2  2025  4200000.0
1          Austin      Q2  2025  3800000.0
2          Denver      Q2  2025  3500000.0
3    Miami Branch      Q2  2025    72000.0
4   Dallas Branch      Q2  2025    88000.0
5  Phoenix Branch      Q3  2025   188000.0
6   Austin Branch      Q3  2025   128000.0
7   Denver Branch      Q3  2025   126000.0


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # No token required
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [20]:
def answer_question(user_query):
    query_embedding = embedder.encode(user_query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)[0]
    context = "\n".join([corpus[hit['corpus_id']] for hit in hits])

    prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question:
{user_query}

Answer:"""

    response = chatbot(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
    return response.split("Answer:")[-1].strip()

In [21]:
import gradio as gr

gr.Interface(fn=answer_question, inputs="text", outputs="text",
             title="📊 Chatbot using Mistral LLM",
             description="Ask questions about your enterprise data stored in SQLite.").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d4fcc6eeb1e4af01f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
